# Distributed Alignment Search

In [1]:
from typing import Any

import torch
import nnsight
from nnsight import AbstractModel, LanguageModel, util
# from nnsight.Module import Module
Module = nnsight.Module
from nnsight.toolbox.optim.lora import LORA
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset as hf_Dataset
from das_utils import factual_sampler

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
model = LanguageModel('EleutherAI/pythia-2.8b', device_map="cuda:0")

In [13]:
n_tokens = 10
epochs = 1
answer = "Paris"
answer_tokens = model.tokenizer(answer)
answer_token = answer_tokens["input_ids"][0]

lora = LORA(model.transformer.h[0].mlp, 10)

optimizer = torch.optim.AdamW(lora.parameters(), lr=.1)
dataset = [[" ".join(["_"] * n_tokens), answer_token]] * 100
dataloader = DataLoader(dataset, batch_size=10)


lossfn = util.cross_entropy_loss

In [16]:
for epoch in range(epochs):
    print(epoch)

    for i, (inputs, targets) in enumerate(dataloader):
        print(f"  {i}")

        optimizer.zero_grad()

        with model.forward(inference=False) as runner:
            with runner.invoke(inputs) as invoker:

                lora()
    
                logits = model.lm_head.output.save()

        print(lora.WA)
        loss = lossfn(logits.value[:, -1], targets)
        print(loss)

        loss.backward()

        optimizer.step()

0
  0
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], requires_grad=True)
tensor(nan, grad_fn=<MeanBackward1>)
  1
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], requires_grad=True)
tensor(nan, grad_fn=<MeanBackward1>)
  2
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

In [23]:
with model.generate() as generator:
    with generator.invoke(dataset[0][0]) as invoker:
        pass

print(model.tokenizer.decode(generator.output[0]))


with model.generate() as generator:
    with generator.invoke(dataset[0][0]) as invoker:
        lora()

print(model.tokenizer.decode(generator.output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _!


In [4]:
raw_prealign = factual_sampler(model.tokenizer, 5000, game="pricing_tag")
prealign_dataset = hf_Dataset.from_dict(
    {"input_ids": raw_prealign[0], "labels": raw_prealign[1]})
prealign_dataset.set_format('torch', columns=['input_ids','labels'])
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [5]:
prealign_input_batches = torch.split(prealign_dataset["input_ids"], int(5000/100), dim=0)
prealign_labels_batches = torch.split(prealign_dataset['labels'][:,-1], int(5000/100), dim=0)

In [24]:
with model.generate(max_new_tokens=10) as generator:
    with generator.invoke(batch) as invoker:
        pass

out = generator.output[:,-10:].cpu()
model.tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Yes\n\n### Task:\n\n### Input'

In [9]:
per_batch_acc = []
per_batch_res = []
total_correct = 0

for i, batch in enumerate(prealign_input_batches):
    with model.generate(max_new_tokens=1) as generator:
        with generator.invoke(batch) as invoker:
            pass
            
    out = generator.output[:,-1].cpu()
    per_batch_res.append(out)
    correct = (out==prealign_labels_batches[0]).sum()
    per_batch_acc.append(correct)
    total_correct += (correct)
    

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


KeyboardInterrupt: 